# Laboratorio 2022-2023

##  Sesión 22: Método de Kasiski

¿Se puede intentar descifrar un cifrado de Vigenère si no se tiene información sobre la longitud de la clave, ni se conoce ningún detalle del texto original? A lo largo de esta sesión veremos que sí, con el llamado método de Kasiski.

Durante siglos se creyó que el cifrado de Vigenère era irrompible. Se sabe que Charles Babbage rompió algunas de sus variantes hacia 1854, pero nunca publicó sus resultados al respecto. Fue Friedrich W. Kasiski, un oficial militar prusiano, el primero en publicar, en 1863, una explicación detallada de cómo romper el cifrado de Vigenère que no dependía de ningún conocimiento previo ni del texto original ni de la clave. 

Aunque hemos igualado las frecuencias de las letras en el texto cifrado usando un desplazamiento diferente para cada letra, la seguridad del cifrado de Vigenère tiene una debilidad: la clave se repite. Cuando se encuentran n-gramas (palabras de longitud n) repetidos en el texto cifrado con n$\ge$3, lo más probable es que esos n-gramas sean también iguales en el texto original. Esto implica que han sido cifrados con la misma sustitución, con lo que la distancia entre los comienzos de dos n-gramas iguales dentro del texto cifrado será un múltiplo del periodo que se usó para cifrar el texto. Si el texto es largo, suelen encontrarse varias parejas de n-gramas repetidos con varias distancias que los separan. En estos casos, la longitud de la clave  que se tomó para cifrar el texto será un divisor común de todas las distancias que separen n-gramas iguales, con lo que las posibilidades se reducen bastante.

### **Ejercicio** ###

Evalúa la siguiente celda. Al hacerlo, tendrás en la variable $\tt kasiski$ un texto que hemos interceptado, y que sabemos que ha sido codificado usando el método de Vigenère. ¡Descífralo, y encuentra la clave con la que fue codificado!

Algunas pistas:

- Empieza por inspeccionar el texto codificado y, como hicimos en la sesión anterior, elabora una conjetura sobre qué alfabeto se usó, si quedaron caracteres del texto original sin cifrar o no, etcétera.

- Tu estrategia debe ser realizar un análisis de Kasiski para decidir cuál puede ser la longitud de la clave. Una vez sepas cuál es la longitud de la clave, habrá que proceder como en la sesión anterior. Por supuesto, algunas de las funciones que usaste la sesión anterior te serán de utilidad de nuevo.

- *Sugerencia*: para el análisis de Kasiski,  define una función ${\tt ngramas}({\tt texto,n,m=30}) $ que, dado un texto, devuelva las m cadenas de longitud n que más se repiten en el texto, junto con el número de veces que se repiten, ordenadas de mayor a menor número de apariciones.

- Define también una función ${\tt MCDdistancias}({\tt texto,ngrama})$ que devuelva el MCD de las distancias que separan dos apariciones consecutivas del n-grama $\tt ngrama$ dentro del texto $\tt texto$. 

Tu candidata a ser la longitud de la clave debe deducirse de la información que te proporciona la separación entre n-gramas especialmente frecuentes *que estén formados enteramente por caracteres que hayan sido codificados*.

In [2]:
load('mensaje.py')
len(kasiski)

138174

In [3]:
# Funciones de la sesión anterior

def contraVigenere(clave,alfabeto):
    
    # Compruebo si la clave es válida (todas sus letras están en el alfabeto)
    if any([letra not in alfabeto for letra in clave]): 
        print('La clave y el alfabeto son incompatibles.')
        return None
    
    # Traduzco la clave a números
    clavenum=[alfabeto.index(letra) for letra in clave]
    
    return ''.join([alfabeto[-k] for k in clavenum]) # Construyo la contraclave y la devuelvo

def cifradoVigenere(texto,clave,alfabeto):
    
    # Compruebo si la clave es válida (todas sus letras están en el alfabeto)
    if any([letra not in alfabeto for letra in clave]): 
        print('La clave y el alfabeto son incompatibles.')
        return None

    L=len(alfabeto) # longitud alfabeto
    long=len(clave) # longitud clave
    clavenum=[alfabeto.index(letra) for letra in clave] # Obtengo la versión numérica de la clave
    
    # Paso al cifrado propiamente dicho
    textocif=''
    for k in xsrange(len(texto)):
        letra=texto[k]
        if letra in alfabeto:
            letra=alfabeto[(alfabeto.index(letra)+clavenum[k%long])%L]           
        textocif+=letra
    return textocif
    
def caracteres(texto):
    lista=list(set(texto)) # Usamos set para eliminar repeticiones
    lista.sort()           # Lista ordenada, de menor a mayor
    return ''.join(lista)

def masfrecuentes(long,texto,alfabeto):
    palabra=''
    for j in xsrange(long):
        letras=texto[j::long]                        # Lista de letras en posiciones congruentes con j módulo long
        letrasUA=set(letras).intersection(alfabeto)  # Estas son las letras que se han usado y que están en el alfabeto
        pares=[(letras.count(letra),letra) for letra in letrasUA] # Cuento cuántas veces aparece cada una de las letras usadas del alfabeto
        pares.sort(reverse=true)
        palabra+=pares[0][1]
    return palabra

def contraclave_Vigenere(texto,alfabeto,palabra):
    long=len(palabra) # longitud de la clave
    palabracodif=masfrecuentes(long,texto,alfabeto) # palabra formada por la letra más frecuente de cada bloque en el texto codificadoe
    contra=[]         # cadena en la que concatenaremos los caracteres de la contraclave
    for j in xsrange(long):
        nuevaposicion_masfrec=alfabeto.index(palabracodif[j]) # índice de la letra más frecuente en el bloque j en el texto codificado
        posicion_masfrec=alfabeto.index(palabra[j])           # índice de la letra más frecuente en el bloque j en el texto original
        contra+=alfabeto[(-nuevaposicion_masfrec+posicion_masfrec)] # siguiente carácter en la contraclave (posición j)
    return contra# la contraclave

In [4]:
print(kasiski[:200])

l¿ 7éú¿ ñs 25Íg!
 
"EQJQG1R JUD J72J¡G DE ;BJ¡y 
".(y5r¿v 8CF1 
(óúé7ú'(d!v GTcDBíIE 
óóó2aóñ)n P?DïH 
Í7)!ñlóéu LG9?úNA 
óóy.9üv ;T75HüMO 
ñ¿"!lï,u T;P Í; ,FNHDH¡ 
á&8!íóop HEH7DSÑ 
ú(ï'¿ás 
1ú)Ñyïo(


In [7]:
#Veamos la cadena de caracteres distintos (y ordenados) que aparecen en el texto cifrado 
# usando para ello la función "caracteres" construida en la sesión anterior
caracteres(kasiski)

'\n !"&\'(),-.125789:;?ABCDEFGHIJLMNOPQRSTUVWY[]abcdefghijlmnopqrstuvwxyz¡¿ÉÍÑÚáéíïñóúü'

In [8]:
print(caracteres(kasiski))


 !"&'(),-.125789:;?ABCDEFGHIJLMNOPQRSTUVWY[]abcdefghijlmnopqrstuvwxyz¡¿ÉÍÑÚáéíïñóúü


In [5]:
# load("FuncVigenere.py")
# contraVigenere??

In [9]:
# Defino una función que detecta cuántas veces se repite cada caracter en un texto.
# Por abreviar utilizo el método .count()

def repeticiones(texto):
    letras=set(texto)
    lista=[]
    for letra in letras:
        lista+=[(texto.count(letra), letra)]#Directamente relleno una lista con pares (frecuencia,letra)
    lista.sort(reverse=true)#Ordeno la lista anterior de mayor a menor frecuencia
    return lista

In [ ]:
repeticiones(kasiski)[:20]

[(53924, ' '),
 (3857, '\n'),
 (3547, 'D'),
 (3391, 'E'),
 (3228, 'C'),
 (3045, 'H'),
 (2999, 'N'),
 (2992, 'L'),
 (2967, 'M'),
 (2713, 'G'),
 (2480, '7'),
 (2474, ';'),
 (2300, 'O'),
 (2275, '?'),
 (2123, 'Q'),
 (2061, 'I'),
 (2034, 'J'),
 (2028, 'F'),
 (1945, 'R'),
 (1780, 'T')]

Los caracteres más repetidos son el espacio en blanco y el salto de línea: parece que se confirma que no se han codificado.  Así que esta es nuestra propuesta de alfabeto:

In [11]:
# Ponemos tres comillas para delimitar la cadena del alfabeto porque este contiene la comilla y la comilla doble

alfabeto='''!"&'(),-.125789:;?ABCDEFGHIJLMNOPQRSTUVWY[]abcdefghijlmnopqrstuvwxyz¡¿ÉÍÑÚáéíïñóúü''' 

In [12]:
def ngramas(texto,n,m=30):
    diccio=dict()#Será un diccionario de frecuencias de los n-gramas de texto
    for k in xsrange(len(texto)-n+1): # k es la posición de la letra de comienzo del n-grama
        ngrama=texto[k:k+n]           # Este es el n-grama que comienza en la posición k 
        if ngrama not in diccio: 
            diccio[ngrama]=1
        else: 
            diccio[ngrama]+=1 
    lista=[(diccio[ngrama], ngrama) for ngrama in diccio]
    lista.sort(reverse=true)
    lista2=[(ngr,frec) for frec,ngr in lista]#Lista de pares (ngrama,frecuencia)
    return lista2[:m]#Solo quiero los m más frecuentes           

In [13]:
print(ngramas(kasiski,3))

[('   ', 33348), ('\n  ', 3035), ('p\n ', 254), ('u\n ', 216), ('  S', 161), ('x\n ', 159), (' SO', 151), (' JL', 145), ('  J', 145), ('  7', 145), ('  P', 141), ('SO8', 139), ('O\n ', 137), ('LD ', 136), (' Y ', 134), (' GT', 133), ('O8 ', 129), (' JU', 128), ('HW;', 125), ('  C', 125), ('JUD', 124), (' S ', 124), ('UD ', 122), (' HW', 121), (' QT', 120), ('JLD', 119), (' 8G', 119), (' 7D', 119), ('y  ', 117), ('PMG', 117)]


In [15]:
def MCDdistancias(texto,ngrama):
    #Lista de posiciones del ngrama en el texto
    posiciones=[]
    for k in xsrange(len(texto)-len(ngrama)+1):  
        if texto[k:k+len(ngrama)]==ngrama: # Indentifico en qué posiciones k gaparece el n-grama
            posiciones.append(k)           # Las añado a la lista posiciones
    #Lista de distancias entre dos apariciones consecutivas
    distancias=[]
    for j in xsrange(len(posiciones)-1):
        distancias.append(posiciones[j+1]-posiciones[j])
    # Solo tiene sentido calcular distancias si el n-grama aparece al menos dos veces
    if len(distancias)>=1: 
        return gcd(distancias)
    else: return 0  # Si no hay al menos dos apariciones el programa devolverá 0 

In [18]:
print(MCDdistancias(kasiski,'\n  '))
print(MCDdistancias(kasiski,'oooo'))
print(MCDdistancias(kasiski,'  S'))
print(MCDdistancias(kasiski,' SO'))
print(MCDdistancias(kasiski,' JL'))

1
0
1
1
1


Ok, analizar 3-gramas en los que intervienen espacios o saltos de línea no es muy útil... Miremos 3-gramas frecuentes que sólo contengan caracteres sí codificados.

In [19]:
print(MCDdistancias(kasiski,'SO8'))
print(MCDdistancias(kasiski,'HW;'))
print(MCDdistancias(kasiski,'JUD'))
print(MCDdistancias(kasiski, u'JLD'))
print(MCDdistancias(kasiski, u'PMG'))

9
9
9
9
9


**¡La sospecha de que la clave es de longitud 9 es MUY FUERTE!**

In [21]:
# Observación: en lugar de ir probando longitudes de ngramas de uno en uno, podemos usar un for
for n in [2..10]:
    distancias=[]
    ngramasfrec=[ngr[0] for ngr in ngramas(kasiski,n)]
    for ngrama in ngramasfrec:
        distancias.append(MCDdistancias(kasiski,ngrama))
    print(distancias)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1, 1, 9, 9, 9, 1, 9, 1, 9, 9, 9, 9, 9, 1, 1, 9]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 9, 1, 9, 9, 1, 1, 1, 9, 9, 3, 1, 9, 1, 1, 9, 9, 9, 9, 9, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 9, 1, 1, 9, 1, 1, 9, 9, 9, 1, 9, 9, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [22]:
#Veamos alguna longitud más de ngramas: n=25
distancias=[]
ngramasfrec=[ngr[0] for ngr in ngramas(kasiski,25)]
for ngrama in ngramasfrec:
    distancias.append(MCDdistancias(kasiski,ngrama))
print(distancias)

[1, 1, 1, 1, 9, 9, 1, 1, 9, 1, 1, 1, 9, 9, 1, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]


Se confirma lo dicho anteriormente ¡con n-gramas de longitud 25!

In [23]:
#Ya tenemos una apuesta: clave de longitud 9
lc=9

In [24]:
contraclave=contraclave_Vigenere(kasiski, alfabeto, 'e'*lc) 

# Ya sabemos que aquí hay incertidumbres
# ¿Sería un texto en castellano?
# Si lo era, ¿será la "e" la más frecuente de cada bloque?

In [25]:
original=cifradoVigenere(kasiski,contraclave,alfabeto)
print(original[:500])

LA VIDA ES SUEÑO
 
Personas que hablan en ella: 
ROSAURA, dama 
SEGISMUNDO, príncipe 
CLOTALDO, viejo 
ESTRELLA, infanta 
CLARÍN, gracioso 
BASILIO, rey de Polonia 
ASTOLFO, infante 
GUARDAS 
SOLDADOS 
MÚSICOS 
 
ACTO PRIMERO
[En las montañas de Polonia]

Salen en lo alto de un monte ROSAURA, en hábito de hombre, de
camino, y en representado los primeros versos va bajando

ROSAURA:   Hipogrifo violento
           que corriste parejas con el viento,
           ¿dónde, rayo sin llama,
           p


In [26]:
clave=contraVigenere(contraclave,alfabeto)
print(clave)

Hipogrifo


In [28]:
#OJO: es bastante largo!!
#Quita el # de la siguiente línea si quieres verlo

# print(original)

**OTRO EJEMPLO:**

In [29]:
load("novela.py")
len(ejemplar)
ngramas(ejemplar, 7)

[(' de la ', 112),
 ('       ', 97),
 (' habia ', 91),
 ('.\n \n --', 85),
 ('lo que ', 79),
 ('huésped', 77),
 (' huéspe', 77),
 (' lo que', 76),
 (' Costan', 68),
 ('a, que ', 67),
 (' que no', 67),
 ('o, que ', 66),
 (' que se', 66),
 ('respond', 65),
 ('que no ', 65),
 ('ostanza', 62),
 ('Costanz', 62),
 (' respon', 62),
 (' que es', 62),
 ('rregido', 60),
 ('regidor', 60),
 ('orregid', 60),
 ('corregi', 60),
 (' correg', 60),
 ('e habia', 58),
 ('vendaño', 56),
 ('espondi', 56),
 ('Avendañ', 56),
 (' porque', 56),
 (' Avenda', 56)]